# Import packages

In [1]:
import MetaTrader5 as mt
import pandas as pd
import ml_collections
import yaml

from datetime import datetime
import sys

sys.path.insert(0, 'D:/Intraday_trading')
from src.support import *

import time
from IPython.display import clear_output

In [2]:
config_dir = 'D:/Intraday_trading/configs/config.yaml'
with open(config_dir) as file:
    config = ml_collections.ConfigDict(yaml.safe_load(file))


In [3]:
acc_config = config.real_acc

# Login account

In [4]:
login_metatrader(acc_config = acc_config)

# Send orders

## instant buy/ sell by market 

In [59]:
def get_market_price(symbol, type):
    '''
    type = 0: ORDER_TYPE_BUY
    type = 1: ORDER_TYPE_SELL
    type = 2: ORDER_TYPE_BUY_LIMIT
    type = 3: ORDER_TYPE_SELL_LIMIT
    type = 4: ORDER_TYPE_BUY_STOP
    type = 5: ORDER_TYPE_SELL_STOP
    '''
    if type == 0 or type == 2 or type == 4:
        return mt.symbol_info(symbol).ask
    elif type == 1 or type == 3 or type == 5:
        return mt.symbol_info(symbol).bid


def get_direction_multiplier(type):
    if type == 0 or type == 2 or type == 4:
        return 1
    elif type == 1 or type == 3 or type == 5:
        return -1


def get_profit_multiplier(symbol):
    if symbol[0:3] == 'BTC':
        return 100
    elif symbol[0:3] == 'XAU':
        return 10


def send_order(symbol, action, volume, type, 
                 price, 
                 sl, tp, deviation, 
                 comment, type_time, type_filling,
                 ref_profit = 20,
                 ref_loss = 10
                 ):
    
    '''
    auto set up desired profit and loss if not provided sl, tp level: profit 20 usd and loss 10 usd
    type = 0: ORDER_TYPE_BUY
    type = 1: ORDER_TYPE_SELL
    type = 2: ORDER_TYPE_BUY_LIMIT
    type = 3: ORDER_TYPE_SELL_LIMIT
    type = 4: ORDER_TYPE_BUY_STOP
    type = 5: ORDER_TYPE_SELL_STOP

    action = 1: TRADE_ACTION_DEAL
    action = 5: TRADE_ACTION_PENDING
    '''

    if symbol is None:
        raise AssertionError('Missing symbol!')
    
    if action is None:
        raise AssertionError('Missing action!')
        
    if volume is None:
        volume = 0.01

    if type is None:
        raise AssertionError('Missing action type (buy/ sell by market/ stop/ limit)!')
    elif (type == 0 or type == 1) and (action != 1):
        raise AssertionError('Action and type are not valid. May be the action = TRADE_ACTION_DEAL')
    elif (type == 2 or type == 3 or type == 4 or type == 5) and (action != 5):
        raise AssertionError('Action and type are not valid. May be the action = TRADE_ACTION_PENDING')    

    if (price is None) and (action == 1):
        price = get_market_price(symbol, type)
    elif (price is None) and (action == 5):
        raise AssertionError('Action and price are not valid. Price should not be null in PENDING orders')    
    elif (price is not None) and (action == 5) and (type == 2 or type == 5) and (price > get_market_price(symbol, type)):
        raise AssertionError('Price is not valid. Input price should be LOWER than current asset price in BUY LIMIT/ SELL STOP orders')
    elif (price is not None) and (action == 5) and (type == 3 or type == 4) and (price < get_market_price(symbol, type)):
        raise AssertionError('Price is not valid. Input price should be HIGHER than current asset price in BUY STOP/ SELL LIMIT orders')
    
    
    direction_multiplier = get_direction_multiplier(type)
    profit_multiplier = get_profit_multiplier(symbol)
    
    if sl is None:
        sl = price - mt.symbol_info(symbol).point * direction_multiplier * ref_loss * profit_multiplier / volume

    if tp is None:
        tp = price + mt.symbol_info(symbol).point * direction_multiplier * ref_profit * profit_multiplier / volume
    
    if type_time is None:
        type_time = mt.ORDER_TIME_GTC

    if type_filling is None:
        type_filling = mt.ORDER_FILLING_FOK
    
    
    request = {
        "action": action,
        "symbol": symbol,
        "volume": volume,
        "type": type,
        "price": price,
        "sl": sl,
        "tp": tp,
        "deviation": deviation,
        "comment": comment,
        "type_time": type_time, # mt.ORDER_TIME_GTC, # mt.ORDER_TIME_DAY
        "type_filling": type_filling,
    }

    # send a trading request
    result = mt.order_send(request)

    print(result)

    if result.retcode != mt.TRADE_RETCODE_DONE:
        print("order_send failed, retcode={}".format(result.retcode))
        # request the result as a dictionary and display it element by element
        result_dict=result._asdict()
        for field in result_dict.keys():
            print("   {}={}".format(field,result_dict[field]))
            # if this is a trading request structure, display it element by element as well
            if field=="request":
                traderequest_dict=result_dict[field]._asdict()
                for tradereq_filed in traderequest_dict:
                    print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))
        


    return(request, result)

In [60]:
symbol = 'BTCUSDm'

action = mt.TRADE_ACTION_DEAL
volume = 0.03
type = mt.ORDER_TYPE_BUY
deviation = 20
comment = 'python script open'
type_time = mt.ORDER_TIME_GTC # mt.ORDER_TIME_DAY
type_filling = mt.ORDER_FILLING_FOK

price = None
sl = None
tp = None

In [61]:
order_sent = send_order(symbol, 
                    action, 
                    volume, 
                    type, 
                    price, 
                    sl, 
                    tp, 
                    deviation = deviation, 
                    comment = comment, 
                    type_time = type_time, 
                    type_filling = type_filling,
                    ref_profit = 20,
                    ref_loss = 10)

OrderSendResult(retcode=10009, deal=1117180227, order=1461622811, volume=0.03, price=97745.44, bid=97716.64, ask=97745.44, comment='python script op', request_id=2292176609, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='BTCUSDm', volume=0.03, price=97745.97, stoplimit=0.0, sl=97412.63666666667, tp=98412.63666666667, deviation=20, type=0, type_filling=0, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))


## buy/ sell stop

In [68]:
symbol = 'BTCUSDm'

action = mt.TRADE_ACTION_PENDING
volume = 0.03
type = mt.ORDER_TYPE_SELL_LIMIT
deviation = 20
comment = 'python script open'
type_time = mt.ORDER_TIME_GTC # mt.ORDER_TIME_DAY
type_filling = mt.ORDER_FILLING_FOK

price = get_market_price(symbol, type) + 2000
    
sl = None
tp = None

In [69]:
order_sent = send_order(symbol, 
                    action, 
                    volume, 
                    type, 
                    price, 
                    sl, 
                    tp, 
                    deviation = deviation, 
                    comment = comment, 
                    type_time = type_time, 
                    type_filling = type_filling,
                    ref_profit = 20,
                    ref_loss = 10)

OrderSendResult(retcode=10009, deal=0, order=1461623888, volume=0.03, price=0.0, bid=97666.84, ask=97695.64, comment='python script op', request_id=2292176612, retcode_external=0, request=TradeRequest(action=5, magic=0, order=0, symbol='BTCUSDm', volume=0.03, price=99666.36, stoplimit=0.0, sl=99999.69333333333, tp=98999.69333333333, deviation=20, type=3, type_filling=0, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))


## (partial) close order

In [35]:
# close position

positions = mt.positions_get()
print('open positions', positions)

# Working with 1st position in the list and closing it
pos1 = positions[0]


open positions (TradePosition(ticket=52038394064, time=1737539077, time_msc=1737539077176, time_update=1737539077, time_update_msc=1737539077176, type=0, magic=0, identifier=52038394064, reason=3, volume=0.01, price_open=1.04185, sl=1.03185, tp=1.06185, price_current=1.0415, swap=0.0, profit=-0.35, symbol='EURUSD', comment='python script op', external_id=''),)


In [26]:

def reverse_type(type):
    # to close a buy positions, you must perform a sell position and vice versa
    if type == mt.ORDER_TYPE_BUY:
        return mt.ORDER_TYPE_SELL
    elif type == mt.ORDER_TYPE_SELL:
        return mt.ORDER_TYPE_BUY


def get_close_price(symbol, type):
    if type == mt.ORDER_TYPE_BUY:
        return mt.symbol_info(symbol).bid
    elif type == mt.ORDER_TYPE_SELL:
        return mt.symbol_info(symbol).ask

request = {
    "action": mt.TRADE_ACTION_DEAL,
    "position": pos1.ticket,
    "symbol": pos1.symbol,
    "volume": 0.02,
    "type": reverse_type(pos1.type),
    "price":get_close_price(pos1.symbol, pos1.type),
    "deviation": 20,
    "magic": 0,
    "comment": "python close order",
    "type_time": mt.ORDER_TIME_GTC,
    "type_filling": mt.ORDER_FILLING_IOC,  # some brokers accept mt.ORDER_FILLING_FOK only
}

result = mt.order_send(request)
result

OrderSendResult(retcode=10009, deal=51993956094, order=52038275533, volume=0.02, price=2749.48, bid=2749.48, ask=2749.62, comment='Request executed', request_id=1928057471, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='XAUUSD', volume=0.02, price=2749.48, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=1, type_filling=1, type_time=0, expiration=0, comment='python close order', position=52038247240, position_by=0))

# Deals monitor 

In [10]:
from_date=datetime(2025,1,1)
to_date=datetime.now()

deals = mt.history_deals_get(from_date, to_date, group="XAUUSD")

# df_deals = pd.DataFrame(deals, columns = deals[0]._asdict().keys())

# df_deals['time'] = pd.to_datetime(df_deals['time'], unit = 's')
# df_deals['time_msc'] = pd.to_datetime(df_deals['time_msc'], unit = 'ms')

# df_deals['VN_time'] = df_deals['time'] + pd.Timedelta('7 hours')
# df_deals['VN_time_msc'] = df_deals['time_msc'] + pd.Timedelta('7 hours')


In [11]:
deals

()

In [6]:
ref_deals_entry = pd.read_csv('D:/Intraday_trading/src/ref_tables/ref_deals_entry.csv', encoding_errors = 'ignore')
ref_deals_reason = pd.read_csv('D:/Intraday_trading/src/ref_tables/ref_deals_reason.csv', encoding_errors = 'ignore')
ref_deals_type = pd.read_csv('D:/Intraday_trading/src/ref_tables/ref_deals_type.csv', encoding_errors = 'ignore')


In [7]:
df_deals = df_deals.merge(ref_deals_entry, how = 'left', left_on = 'type', right_on = ref_deals_entry.index)
df_deals = df_deals.merge(ref_deals_reason, how = 'left', left_on = 'type', right_on = ref_deals_reason.index)
df_deals = df_deals.merge(ref_deals_type, how = 'left', left_on = 'type', right_on = ref_deals_type.index)

In [8]:
df_deals.tail(6)

,ticket,order,time,time_msc,type,entry,magic,position_id,reason,volume,price,commission,swap,profit,fee,symbol,comment,external_id,VN_time,VN_time_msc,IDENTIFIER_ENTRY,DESCRIPTION_ENTRY,IDENTIFIER_REASON,DESCRIPTION_REASON,IDENTIFIER_TYPE,DESCRIPTION_TYPE
115,83636196,302562186,2024-06-28 07:33:08,2024-06-28 07:33:08.691,0,0,0,302562186,1,0.03,2325.098,0.0,0.0,0.00,0.0,XAUUSDm,,,2024-06-28 14:33:08,2024-06-28 14:33:08.691,DEAL_ENTRY_IN,Entry in,DEAL_REASON_CLIENT,The deal was executed as a result of activatio...,DEAL_TYPE_BUY,Buy
116,83663964,302683525,2024-06-28 08:38:29,2024-06-28 08:38:29.924,0,0,0,302683525,1,0.03,2331.499,0.0,0.0,0.00,0.0,XAUUSDm,,,2024-06-28 15:38:29,2024-06-28 15:38:29.924,DEAL_ENTRY_IN,Entry in,DEAL_REASON_CLIENT,The deal was executed as a result of activatio...,DEAL_TYPE_BUY,Buy
117,83718903,302801066,2024-06-28 11:02:45,2024-06-28 11:02:45.553,0,0,0,302801066,1,0.03,2337.034,0.0,0.0,0.00,0.0,XAUUSDm,,,2024-06-28 18:02:45,2024-06-28 18:02:45.553,DEAL_ENTRY_IN,Entry in,DEAL_REASON_CLIENT,The deal was executed as a result of activatio...,DEAL_TYPE_BUY,Buy
118,83720770,302808511,2024-06-28 11:06:36,2024-06-28 11:06:36.043,0,0,0,302808511,1,0.03,2336.663,0.0,0.0,0.00,0.0,XAUUSDm,,,2024-06-28 18:06:36,2024-06-28 18:06:36.043,DEAL_ENTRY_IN,Entry in,DEAL_REASON_CLIENT,The deal was executed as a result of activatio...,DEAL_TYPE_BUY,Buy
119,83855207,303120409,2024-06-28 14:43:15,2024-06-28 14:43:15.855,1,1,0,302801066,4,0.03,2322.824,0.0,0.0,-42.63,0.0,XAUUSDm,[sl 2322.824],,2024-06-28 21:43:15,2024-06-28 21:43:15.855,DEAL_ENTRY_OUT,Entry out,DEAL_REASON_MOBILE,The deal was executed as a result of activatio...,DEAL_TYPE_SELL,Sell
120,83891728,303202600,2024-06-28 16:05:55,2024-06-28 16:05:55.011,0,0,0,303202600,1,0.03,2330.628,0.0,0.0,0.00,0.0,XAUUSDm,,,2024-06-28 23:05:55,2024-06-28 23:05:55.011,DEAL_ENTRY_IN,Entry in,DEAL_REASON_CLIENT,The deal was executed as a result of activatio...,DEAL_TYPE_BUY,Buy


# Orders monitor

In [12]:
from_date=datetime(2024,6,15)
to_date=datetime.now()
# get deals for symbols whose names contain "XAUUSDm" within a specified interval
orders = mt.history_orders_get(from_date, to_date, group="*XAUUSD*")

df_orders = pd.DataFrame(orders, columns = orders[0]._asdict().keys())
df_orders['time_setup'] = pd.to_datetime(df_orders['time_setup'], unit = 's')
df_orders['VN_time'] = df_orders['time_setup'] + pd.Timedelta('7 hours')

In [13]:
df_orders.tail(6)

,ticket,time_setup,time_setup_msc,time_done,time_done_msc,time_expiration,type,type_time,type_filling,state,magic,position_id,position_by_id,reason,volume_initial,volume_current,price_open,sl,tp,price_current,price_stoplimit,symbol,comment,external_id,VN_time
1441,276764934,2025-03-21 14:37:32,1742567852433,1742567852,1742567852434,0,0,0,0,4,0,276764934,0,1,0.05,0.0,0.0,0.0,0.0,3006.965,0.0,XAUUSDm,,,2025-03-21 21:37:32
1442,276784648,2025-03-21 14:47:27,1742568447184,1742568447,1742568447193,0,0,0,0,4,0,276784648,0,1,0.05,0.0,0.0,0.0,0.0,3013.525,0.0,XAUUSDm,,,2025-03-21 21:47:27
1443,276850375,2025-03-21 15:29:34,1742570974543,1742570974,1742570974544,0,1,0,1,4,0,276764934,0,1,0.05,0.0,0.0,0.0,0.0,3014.340,0.0,XAUUSDm,,,2025-03-21 22:29:34
1444,276850376,2025-03-21 15:29:34,1742570974543,1742570974,1742570974544,0,1,0,1,4,0,276784648,0,1,0.05,0.0,0.0,0.0,0.0,3014.340,0.0,XAUUSDm,,,2025-03-21 22:29:34
1445,276866075,2025-03-21 15:39:48,1742571588886,1742571588,1742571588887,0,1,0,0,4,0,276866075,0,1,0.05,0.0,0.0,0.0,0.0,3013.188,0.0,XAUUSDm,,,2025-03-21 22:39:48
1446,276866919,2025-03-21 15:40:27,1742571627504,1742571627,1742571627505,0,1,0,0,4,0,276866919,0,1,0.05,0.0,0.0,0.0,0.0,3012.356,0.0,XAUUSDm,,,2025-03-21 22:40:27


# Data

## OHLCV interval data

In [9]:
mt.shutdown()

True

In [10]:
login_metatrader(acc_config = acc_config)

In [56]:
# Compute now date
from_date = datetime(2021, 1, 1)
to_date = datetime(2025, 3, 21)


rates = mt.copy_rates_from("XAUUSDm", mt.TIMEFRAME_M1, from_date, 99999)
df_rates = pd.DataFrame(rates)

df_rates["time"] = pd.to_datetime(df_rates["time"], unit="s")



In [57]:
df_rates

,time,open,high,low,close,tick_volume,spread,real_volume
0,2024-12-06 18:45:00,2636.534,2636.593,2636.068,2636.255,47,469,0


In [ ]:
# Compute now date
from_date = datetime(2001, 1, 1)
to_date = datetime(2024, 4, 11)


rates = mt.copy_rates_range("EURUSDm", mt.TIMEFRAME_M1, from_date, to_date)
df_rates = pd.DataFrame(rates)

df_rates["time"] = pd.to_datetime(df_rates["time"], unit="s")
print(df_rates['time'][0])



rates = mt.copy_rates_range("EURUSDm", mt.TIMEFRAME_M5, from_date, to_date)
df_rates = pd.DataFrame(rates)

df_rates["time"] = pd.to_datetime(df_rates["time"], unit="s")
print(df_rates['time'][0])



rates = mt.copy_rates_range("EURUSDm", mt.TIMEFRAME_M15, from_date, to_date)
df_rates = pd.DataFrame(rates)

df_rates["time"] = pd.to_datetime(df_rates["time"], unit="s")
print(df_rates['time'][0])



rates = mt.copy_rates_range("EURUSDm", mt.TIMEFRAME_H1, from_date, to_date)
df_rates = pd.DataFrame(rates)

df_rates["time"] = pd.to_datetime(df_rates["time"], unit="s")
print(df_rates['time'][0])



rates = mt.copy_rates_range("EURUSDm", mt.TIMEFRAME_D1, from_date, to_date)
df_rates = pd.DataFrame(rates)

df_rates["time"] = pd.to_datetime(df_rates["time"], unit="s")
print(df_rates['time'][0])

## Ticks data

In [ ]:
# https://thiagomarzagao.com/2021/05/12/mt5/

In [ ]:
# Compute now date
from_date = datetime(2020, 4, 10)
to_date = datetime(2024, 4, 11)

# Extract n Ticks before now
ticks = mt.copy_ticks_range("EURUSDm", from_date, to_date, mt.COPY_TICKS_ALL)

# Transform Tuple into a DataFrame
df_ticks = pd.DataFrame(ticks)

# Convert number format of the date into date format
df_ticks["time"] = pd.to_datetime(df_ticks["time"], unit="s")


df_ticks